In [1]:
!pip install gdown

In [2]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import gdown
import zipfile

2023-11-12 16:02:32.186068: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-12 16:02:32.895334: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
url = 'https://drive.google.com/uc?id=1O4YR4UBatOLnaP4gMHbmFw7UJvhhxFwq&export=download'
gdown.download(url,quiet=False)

Downloading...
From (uriginal): https://drive.google.com/uc?id=1O4YR4UBatOLnaP4gMHbmFw7UJvhhxFwq&export=download
From (redirected): https://drive.google.com/uc?id=1O4YR4UBatOLnaP4gMHbmFw7UJvhhxFwq&export=download&confirm=t&uuid=48e17287-8b19-42a8-8195-bde9e1d2af2a
To: /jupyter/train_data.zip
100%|██████████| 173M/173M [00:30<00:00, 5.72MB/s] 


'train_data.zip'

In [4]:
!pip install patool
import patoolib
patoolib.extract_archive("train_data.zip" , outdir = "train" )

patool: Extracting train_data.zip ...
patool: ... train_data.zip extracted to `train'.


'train'

In [5]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

# 1. Data Preparation: Organize your dataset into subdirectories.

train_data_dir = "train/train_data"



In [6]:
!pip install git+https://github.com/qubvel/classification_models.git

  Cloning https://github.com/qubvel/classification_models.git to /tmp/pip-req-build-0e9l1huz
  Running command git clone -q https://github.com/qubvel/classification_models.git /tmp/pip-req-build-0e9l1huz
  Running command git submodule update --init --recursive -q
  Created wheel for image-classifiers: filename=image_classifiers-1.0.0-py3-none-any.whl size=20031 sha256=4bb0579e46c745665641b17ac1ee6c722a52e565295040b4bcce732a1aaf97b5
  Stored in directory: /tmp/pip-ephem-wheel-cache-sbrddu68/wheels/77/23/ad/afd2caf2877de1a21565559524b6435e5396579c14207062f3
Successfully built image-classifiers


In [7]:
import sys
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from classification_models.keras import Classifiers
inceptionv3, preprocess_input = Classifiers.get('inceptionv3')

# Define image dimensions
image_height, image_width = 299, 299

# Define the number of classes (labels)
num_classes = 10

# Define batch size
batch_size = 64

'''
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,  # Use ResNet50's preprocessing
    validation_split = 0.2,
    rescale = 1./255
)
'''
#Create an instance of ImageDataGenerator with preprocessing

train_datagen = ImageDataGenerator(
     preprocessing_function=preprocess_input,
     width_shift_range = 0.2,
     height_shift_range = 0.2,
     validation_split = 0.2
)




# Create data generators for training and validation
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(image_height, image_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(image_height, image_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)






Found 7994 images belonging to 10 classes.
Found 1996 images belonging to 10 classes.


In [8]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping
from tensorflow.keras.optimizers import Adam ,SGD ,RMSprop
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.models import Sequential
from keras.callbacks import LearningRateScheduler
import math
from keras import regularizers
from keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D, GlobalAveragePooling2D, AveragePooling2D

In [10]:
def scheduler(epoch, lr):
    return lr * math.exp(-0.1)

from keras import regularizers
model = inceptionv3(weights='imagenet', include_top=False, input_shape=(image_height, image_width, 3))


for layer in model.layers[:-1]:
    layer.trainable = False
    
x = model.output
x = GlobalAveragePooling2D()(x)
predictions = Dense(num_classes, activation='softmax' , kernel_regularizer=regularizers.l2(0.02))(x)


new_model = keras.models.Model(inputs=model.input , outputs = predictions)

alpha = 0.01  # Learning rate
beta_1 = 0.9   # First moment decay (similar to momentum in SGD)

new_model.compile(optimizer = Adam(learning_rate=alpha, beta_1=beta_1) , loss='categorical_crossentropy', metrics=['acc'])

my_callbacks = [
    tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True , min_delta=0.001),
    tf.keras.callbacks.ModelCheckpoint(filepath='model.{epoch:02d}-{val_loss:.2f}.h5'),
    tf._keras.callbacks.ReduceLROnPlateau(monitor='val_acc', factor= 0.2 ,patience=5, min_lr=0.01 , min_delta=0.01)
]
lr_scheduler = LearningRateScheduler(scheduler)


In [ ]:
new_model.fit(
    train_generator,
    epochs=100,
    validation_data=validation_generator,
    callbacks=[my_callbacks,lr_scheduler]
)
#model.save('my_model.h5')

Epoch 1/100


2023-11-12 16:03:51.459995: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8600
2023-11-12 16:03:51.993499: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-11-12 16:03:52.458136: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:606] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-11-12 16:03:53.386157: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7fb19d1519e0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-11-12 16:03:53.386183: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3080, Compute Capability 8.6
2023-11-12 16:03:53.390172: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-11-12 16:03:53.44599

 47/125 [==========>...................] - ETA: 1:03 - loss: 2.9779 - acc: 0.3444